# Data Preparation

### Import necessary libraries

In [2]:
import re

import pandas as pd

### Load data from the Google Sheet

In [18]:
sheet_id = "1OHl5u6-31KyQSYHeLJBXAk9Xc4eiLPviXIlc6ycaTM4"
page_id = "167370872"

df = pd.read_csv(
    f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?gid=167370872&format=csv"
)

df.to_csv("../data/ltv_raw_data.csv")

df.head()

,user_id,event_timestamp,first_purchase_time,cohort_week,product_id,lifetime_months
0,95287b91-85af-4174-9986-aeafb3853bb6,2023-09-21 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,6
1,95287b91-85af-4174-9986-aeafb3853bb6,2023-03-25 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,0
2,95287b91-85af-4174-9986-aeafb3853bb6,2023-04-24 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,1
3,95287b91-85af-4174-9986-aeafb3853bb6,2023-05-24 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,2
4,95287b91-85af-4174-9986-aeafb3853bb6,2023-06-23 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,3


### Check data for a consistency, NaN values, fix typos in column names

In [19]:
df.rename(columns={"lifetime_monhts": "lifetime_months"}, inplace=True)
df.columns

Index(['user_id', 'event_timestamp', 'first_purchase_time', 'cohort_week',
       'product_id', 'lifetime_months'],
      dtype='object')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   user_id              1200 non-null   object
 1   event_timestamp      1200 non-null   object
 2   first_purchase_time  1200 non-null   object
 3   cohort_week          1200 non-null   object
 4   product_id           1200 non-null   object
 5   lifetime_months      1200 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 56.4+ KB


In [21]:
# change event_timestamp, first_purchase_time and cohort_week type to datetime

df[["event_timestamp", "first_purchase_time", "cohort_week"]] = df[
    ["event_timestamp", "first_purchase_time", "cohort_week"]
].apply(pd.to_datetime)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              1200 non-null   object        
 1   event_timestamp      1200 non-null   datetime64[ns]
 2   first_purchase_time  1200 non-null   datetime64[ns]
 3   cohort_week          1200 non-null   datetime64[ns]
 4   product_id           1200 non-null   object        
 5   lifetime_months      1200 non-null   int64         
dtypes: datetime64[ns](3), int64(1), object(2)
memory usage: 56.4+ KB


In [22]:
# check uniqueness for product_id

df["product_id"].unique()

array(['monthly.12.49'], dtype=object)

In [32]:
price = df["product_id"].apply(
    lambda x: (
        float(re.search(r"\d+\.\d+", x).group()) if re.search(r"\d+\.\d+", x) else 0
    )
)
price = price.unique()[0]
df["price"] = price

user_revenue = df.groupby("user_id")["price"].sum()
df["revenue"] = df["user_id"].map(user_revenue)

frequency = df.groupby("user_id").size()
df = df.merge(frequency.rename("frequency"), on="user_id")

df.describe()

,event_timestamp,first_purchase_time,cohort_week,lifetime_months,price,frequency,revenue
count,1200,1200,1200,1200.000000,1.200000e+03,1200.000000,1200.000000
mean,2023-05-16 22:08:55.034166528,2023-04-03 23:56:55.034166784,2023-03-30 10:40:48,1.430833,1.249000e+01,3.773333,47.128933
min,2023-03-06 04:55:30,2023-03-06 04:55:30,2023-03-05 00:00:00,0.000000,1.249000e+01,1.000000,12.490000
25%,2023-04-05 11:13:16.500000,2023-03-17 08:11:44,2023-03-12 00:00:00,0.000000,1.249000e+01,1.000000,12.490000
50%,2023-05-02 05:23:36.500000,2023-03-31 09:34:10.500000,2023-03-26 00:00:00,1.000000,1.249000e+01,4.000000,49.960000
75%,2023-06-18 15:04:00.249999872,2023-04-22 06:10:08,2023-04-16 00:00:00,2.000000,1.249000e+01,6.000000,74.940000
max,2023-10-14 06:00:06,2023-05-07 14:00:39,2023-04-30 00:00:00,7.000000,1.249000e+01,8.000000,99.920000
std,NaN,NaN,NaN,1.787329,3.554195e-15,2.411805,30.123443


In [33]:
# check how many unique values there are in each column

df.nunique()

user_id                 561
event_timestamp        1198
first_purchase_time     559
cohort_week               9
product_id                1
lifetime_months           8
price                     1
frequency                 8
revenue                   8
dtype: int64

### Save cleaned data as a CSV file

In [34]:
df.to_csv("../data/ltv_cleaned_data.csv")